In [1]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

# Wrangling
import pandas as pd
import numpy as np

# Exploring
import scipy.stats as stats

# Visualizing
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# default pandas decimal number display format
pd.options.display.float_format = '{:20,.2f}'.format

import acquire
import summarize as s
import prepare as p

# ignore warnings
import warnings
warnings.filterwarnings("ignore")



# Zillow


For the following, iterate through the steps you would take to create functions: Write the code to do the following in a jupyter notebook, test it, convert to functions, then create the file to house those functions.

You will have a zillow.ipynb file and a helper file for each section in the pipeline.

#### Acquire and Summarize

1) Acquire data from the cloud database.

You will want to end with a single dataframe. Include the logerror field and all other fields related to the properties that are available. You will end up using all the tables in the database.

Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid. - Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction. (Hint: read the docs for the .duplicated method) - Only include properties that have a latitude and longitude value.

2) Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

3) Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

In [2]:
#acquiring data
df = pd.read_csv('zillow_all.csv')

In [3]:
p.df_described(df)

TypeError: tuple indices must be integers or slices, not tuple

In [ ]:
pd.DataFrame(df.T.iloc[:])

In [ ]:
p.flip(df)

In [ ]:
# confirm no duplicates
len(df)-len(df.drop_duplicates())

In [ ]:
#make sure transactions are in 2017
df.transactiondate.max(), df.transactiondate.min()

2) Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)


In [ ]:
s.summarize(df)

In [ ]:
df_info = pd.DataFrame(df.info())

In [ ]:
df.isnull().any(axis=1).sum() 


3) Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

- each row attribute name
- first column number of rows (of original df) with missing values
- second column percent total columns with missing values for this attribute

Not transforming original df, making a new df describing the original df


In [ ]:
def df_described(df):
    df_described = pd.DataFrame({
                                    0: df.columns,
                                    1: df.isnull().sum(axis=0),
                                    2: df.isnull().any(axis=1).sum(), 
                                    3: df.shape[1] - df.count(axis=1),
                                index = [df.columns]
                                   } )
    return df_described

In [ ]:
df = pd.DataFrame(
      [[4, 7, 10],
       [5, 8, 11],
       [6, 9, 12]],
      index=[1, 2, 3],
      columns=['a', 'b', 'c'])

df = pd.DataFrame(
        [[df.columns],
        [df.isnull().any(axis=1).sum()],
        [],]


)

In [ ]:
df_transposed.drop(df_transposed.columns[0], axis = 1, inplace = True)


In [ ]:
df_transposed

def transform_df(df):
    df = pd.DataFrame(df.T.iloc[:,2])
    df.drop(df.columns[0], axis = 1, inplace = True)
    df[0] = df.isnull().sum(axis=0)
    return df
    

In [ ]:
df_transposed[0] = df.isnull().sum(axis=0)

In [ ]:
df_transposed['1'] = 0

In [ ]:
df_transposed

In [ ]:
#second column percent total columns with missing values for this attribute
df_transposed

total columns.count - column[:1]

#### Prepare

1) Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.

2) Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).

    - The input:
        - A dataframe
        - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
        - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
    - The output:
        - The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
    - hint:
        - Look up the dropna documentation.
        - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.
3) Encapsulate your work inside of functions in a wrangle_zillow.py module.

# Mall Customers

Notebook

1) Acquire data from the customers table in the mall_customers database.
2) Summarize the data (include distributions and descriptive statistics).
3) Detect outliers using IQR.
4) Split data into train, validate, and test.
5) Encode categorical columns using a one hot encoder (pd.get_dummies).
6) Handles missing values.
7) Scaling

Encapsulate your work in a wrangle_mall.py python module.
